# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [9]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [10]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 26 compute_cascadelake_r nodes ready to use


### Reserve node

In [11]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [15]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,c967c349-660e-47d2-be35-4fa1491db68c
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.0.200 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-05-03T04:29:46Z
Keypair,trovi-182eabd
Reservation Id,ff7577a5-a7f4-45bc-98f2-c0d80cae3f95
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [16]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.245 port 22.


Connection successful


### Setup nvidia drivers and cuda

Download drivers

In [35]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.67: b'5c20f135fa6b791abdd054e1757e7da1'
  warnings.warn(


Reading package lists...
Building dependency tree...
Reading state information...
Package 'cuda-license-10-1' is not installed, so not removed
Package 'cuda-license-10-2' is not installed, so not removed
Package 'nvidia-egl-wayland-common' is not installed, so not removed
Package 'nvidia-390' is not installed, so not removed
Package 'nvidia-libopencl1-dev' is not installed, so not removed
Package 'nvidia-current' is not installed, so not removed
Package 'nvidia-current-updates' is not installed, so not removed
Package 'nvidia-libopencl1' is not installed, so not removed
Package 'nvidia-driver-410' is not installed, so not removed
Package 'nvidia-387' is not installed, so not removed
Package 'nvidia-387-updates' is not installed, so not removed
Package 'nvidia-experimental-387' is not installed, so not removed
Package 'nvidia-384-updates' is not installed, so not removed
Package 'nvidia-experimental-384' is not installed, so not removed
Package 'nvidia-381' is not installed, so not remo

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2181 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2423 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 c-n-f Metadata [13.6 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3059 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 c-n-f Metadata [624 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [969 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 c-n-f Metadata [21.7 kB]
Get:12 http://security.ubuntu.com/ubun

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libegl-mesa0
  libegl1 libfontenc1 libgbm1 libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0
  libglx-mesa0 libglx0 libice6 libllvm15 libnvidia-cfg1-470
  libnvidia-common-470 libnvidia-compute-470 libnvidia-decode-470
  libnvidia-egl-wayland1 libnvidia-encode-470 libnvidia-extra-470
  libnvidia-fbc1-470 libnvidia-gl-470 libnvidia-ifr1-470 libpciaccess0
  libsensors-config libsensors5 libsm6 libwayland-client0 libx11-xcb1 libxaw7
  libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-shm0
  libxcb-sync1 libxcb-xfixes0 libxcvt0 libxfixes3 libxfont2 libxkbfile1
  libxmu6 libxpm4 libxshmfence1 libxt6 libxxf86vm1 nvidia-compute-utils-470
  nvidia-dkms-470 nvidia-kernel-common-470 nvidia-kernel-source-470
  nvidia-utils-470 x11-common x11-xkb-utils xserver-common xserver-xorg-core
  xserver

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


...

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [2068 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 2595 kB in 1s (2053 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
129 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  ca-certificates-java cuda-command-line-tools-11-2 cuda-compiler-11-2
  cuda-cudart-11-2 cuda-cudart-dev-11-2 cuda-cuobjdump-11-2 cuda-cupti-11-2
  cuda-cupti-dev-11-2 cuda-cuxxfilt-11-2 cuda-documentation-11-2
  cuda-driver-dev-11-2 cuda-gdb-11-2 cuda-libraries-11-2
  cuda-libraries-dev-11-2 cuda-memcheck-11-2 cuda-nsight-11-2
  cuda-nsight-compute-11-2 cuda-nsight-systems-11-2 cuda-nvcc-11-2
  cuda-nvdisasm-11-2 cuda-nvml-dev-11-2 cuda-nvprof-11-2 cuda-nvprune-11-2
  cuda-nvrtc-11-2 cuda-nvrtc-dev-11-2 cuda-nvtx-11-2 cuda-nvvp-11-2
  cuda-samples-11-2 cuda-sanitizer-11-2 cuda-tools-11-2 cuda-visual-tools-11-2
  default-jre default-jre-headless fontconfig-config fonts-dejavu-core
  java-common libavahi-client3 libavahi-common-data libavahi-common3 libcairo2
  libcublas-11-2 libcublas-dev-11-2 libcufft-11-2 libcufft-dev-11-2 libcups2
  libcurand-11-2 l

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 2144 MB in 1min 40s (21.4 MB/s)
Selecting previously unselected package java-common.
(Reading database ... 85211 files and directories currently installed.)
Preparing to unpack .../00-java-common_0.72build2_all.deb ...
Unpacking java-common (0.72build2) ...
Selecting previously unselected package libavahi-common-data:amd64.
Preparing to unpack .../01-libavahi-common-data_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common-data:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-common3:amd64.
Preparing to unpack .../02-libavahi-common3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-client3:amd64.
Preparing to unpack .../03-libavahi-client3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-client3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libcups2:amd64.
Preparing to unpack .../04-libcups2_2.4.1op1-1ubuntu4.11_amd64.deb ...
Unpacking libcups2:amd

<Result cmd="\nsudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && sudo apt-get autoremove -y && sudo apt-get autoclean && sudo rm -rf /usr/local/cuda* && sudo apt update && sudo apt install -y nvidia-driver-470 && wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && sudo dpkg -i cuda-keyring_1.0-1_all.deb && sudo apt update && sudo apt install -y cuda-toolkit-11-2\n" exited=0>

Restart nvidia services

In [36]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

update-initramfs: Generating /boot/initrd.img-5.15.0-124-generic


<Result cmd='sudo modprobe nvidia' exited=0>

Verifiy services

In [37]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

Mon Mar 31 09:32:59 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.256.02   Driver Version: 470.256.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:03:00.0 Off |                    0 |
| N/A   28C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:82:00.0 Off |                    0 |
| N/A   

<Result cmd='nvcc --version' exited=0>

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [19]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

### Run dependencies and first experiment

In [20]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

Reading package lists...
Building dependency tree...
Reading state information...
csvtool is already the newest version (2.4-1build3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


...


Processing Partitions:  10%|█         | 21/201 [29:17<3:19:00, 66.34s/it]

0      0.0
1      1.0
2      1.0
3      0.0
4      1.0
      ... 
145    1.0
146    1.0
147    1.0
148    1.0
149    1.0
Name: 11, Length: 150, dtype: float64
y_true
0      0
1      1
2      1
3      1
4      1
      ..
145    0
146    1
147    1
148    0
149    0
Length: 150, dtype: int64
True: 104 | False: 46
y_pred
0      True
1      True
2      True
3      True
4      True
       ... 
145    True
146    True
147    True
148    True
149    True
Length: 150, dtype: bool
True: 145 | False: 5
prot_attr
0      0.0
1      1.0
2      1.0
3      0.0
4      1.0
      ... 
145    1.0
146    1.0
147    1.0
148    1.0
149    1.0
Length: 150, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  7.769
['1', '80', '2', '9', '12900', '2', '0', '1', '1', '4', '2', '1', '2', '0', '1', '0', '2', '1', '1', '0']
['1', '80', '2', '9', '12900', '2', '0', '1', '1', '4', '2', '0', '2', '0', '1', '0', '2', '1', '1', '0']
[0.06403834] [-0.1282807]
[0.06403828] 


Processing Models: 100%|██████████| 6/6 [1:05:30<00:00, 655.03s/it] 


class_1:  True
class_2:  False
pred1_orig:  0.5253605882563775
pred2_orig:  0.47729588306387755
class_1_orig:  True
class_2_orig:  False
******************
10th column
0      0.0
1      1.0
2      1.0
3      0.0
4      1.0
      ... 
145    1.0
146    1.0
147    1.0
148    1.0
149    1.0
Name: 11, Length: 150, dtype: float64
y_true
0      0
1      1
2      1
3      1
4      1
      ..
145    0
146    1
147    1
148    0
149    0
Length: 150, dtype: int64
True: 104 | False: 46
y_pred
0      True
1      True
2      True
3      True
4      True
       ... 
145    True
146    True
147    True
148    True
149    True
Length: 150, dtype: bool
True: 145 | False: 5
prot_attr
0      0.0
1      1.0
2      1.0
3      0.0
4      1.0
      ... 
145    1.0
146    1.0
147    1.0
148    1.0
149    1.0
Length: 150, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  12.948
['1', '0', '1', '0', '17599', '0', '0', '2', '0', '1', '0', '1', '0', '1', '1', '0

<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>

Results for first experiment

In [21]:
my_server.execute("csvtool readable Fairify/src/GC/age-GC-1.csv | less -S")

Partition_ID Verification SAT_count UNSAT_count UNK_count h_attempt h_success B_compression S_compression ST_compression H_compression T_compression SV-time S-time HV-Time H-Time Total-Time C-check V-accurate Original-acc Pruned-acc Acc-dec C1                                                                                                                                                                                                                               C2
1            unknown      0         0           1         1         0         0.7255        0.0           0.7255         0.0588        0.7843        100.026 100.6  100.034 100.09 200.69     0       0          0.7267       1.0        -                                                                                                                                                                                                                                        
1            unknown      0         0           1         1     

<Result cmd='csvtool readable Fairify/src/GC/age-GC-1.csv | less -S' exited=0>

In [22]:
my_server.execute("csvtool readable Fairify/src/GC/age-GC-8.csv | less -S")

Partition_ID Verification SAT_count UNSAT_count UNK_count h_attempt h_success B_compression S_compression ST_compression H_compression T_compression SV-time S-time HV-Time H-Time Total-Time C-check V-accurate Original-acc Pruned-acc Acc-dec C1                                                                                                                                                                                                                               C2
1            unknown      0         0           1         1         0         0.6471        0.0           0.6471         0.0588        0.7059        100.039 100.63 100.104 100.18 200.81     0       0          0.7467       1.0        -                                                                                                                                                                                                                                        
2            sat          1         0           1         0     

<Result cmd='csvtool readable Fairify/src/GC/age-GC-8.csv | less -S' exited=0>

In [23]:
my_server.execute("csvtool readable Fairify/src/GC/age-synthetic-german-predicted-GC-1-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score  Pruned Accuracy Pruned F1 DI                 SPD                   EOD AOD                  ERD                  CNT          TI
1            0.7266666666666667 0.8353413654618473 1.0             1.0       1.0014025245441796 0.0013553808620222485 0.0 0.005555555555555591 0.021415017619951127 [0.95333333] 0.05594337812248188
1            0.7266666666666667 0.8353413654618473 1.0             1.0       1.0014025245441796 0.0013553808620222485 0.0 0.005555555555555591 0.021415017619951127 [0.95333333] 0.05594337812248188
2            0.7266666666666667 0.8353413654618473 1.0             1.0       1.0014025245441796 0.0013553808620222485 0.0 0.005555555555555591 0.021415017619951127 [0.95333333] 0.05594337812248188
3            0.7266666666666667 0.8353413654618473 1.0             1.0       1.0014025245441796 0.0013553808620222485 0.0 0.005555555555555591 0.021415017619951127 [0.95333333] 0.05594337812248188
4            0.7266666666666667 

<Result cmd='csvtool readable Fairify/src/GC/age-synthetic-german-predicted-GC-1-metrics.csv | less -S' exited=0>

In [25]:
my_server.execute("csvtool readable Fairify/src/GC/age-synthetic-german-predicted-GC-8-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score  Pruned Accuracy Pruned F1 DI                 SPD                   EOD AOD                   ERD                  CNT          TI
1            0.7466666666666667 0.8455284552845529 1.0             1.0       0.9851555809306308 -0.014095960965031185 0.0 -0.016666666666666663 0.005963675792897805 [0.94533333] 0.05440176596075252
2            0.7466666666666667 0.8455284552845529 1.0             1.0       0.9851555809306308 -0.014095960965031185 0.0 -0.016666666666666663 0.005963675792897805 [0.94533333] 0.05440176596075252
3            0.7466666666666667 0.8455284552845529 1.0             1.0       0.9851555809306308 -0.014095960965031185 0.0 -0.016666666666666663 0.005963675792897805 [0.94533333] 0.05440176596075252
4            0.7466666666666667 0.8455284552845529 1.0             1.0       0.9851555809306308 -0.014095960965031185 0.0 -0.016666666666666663 0.005963675792897805 [0.94533333] 0.05440176596075252
5            0.746666666666

<Result cmd='csvtool readable Fairify/src/GC/age-synthetic-german-predicted-GC-8-metrics.csv | less -S' exited=0>